In [1]:
import pandas as pd

from sys import platform

if platform == "win32":
    path = 'C:/Users/olive/GitHub/f1-analytics/'
elif platform == "darwin":
    path = '~/Documents/GitHub/f1-analytics/'
    # path = '/Users/oliverjcarter/Documents/GitHub/f1-analytics/'

In [2]:
def format_time(x):
    if not any(i in x for i in ['DNF', 'DNS']):
        if ':' in x:
            return round(float(str(x).split(':')[1]) + (60 * float(str(x).split(':')[0])), 3) if x != 0 else 0
        else:
            return(round(float(x), 3))
    else:
        return x

In [3]:
races = pd.read_csv(path+'data/races.csv')
qualifying = pd.read_csv(path+'data/qualifying.csv')
constructors = pd.read_csv(path+'data/constructor_standings.csv')
drivers = pd.read_csv(path+'data/driver_standings.csv')
results = pd.read_csv(path+'data/results.csv')

### Redundant until new weather collection method is implemented
# weather = pd.read_csv(path+'data/weather.csv')

In [4]:
laps1 = pd.read_csv(path+'data/laps-2014-2017.csv')
laps2 = pd.read_csv(path+'data/laps-2017-22.csv')

In [5]:
laps = pd.concat([laps1, laps2]).drop_duplicates()

laps.shape

(168937, 6)

In [6]:
laps[laps.duplicated()]

,season,round,lap,driver,position,time


In [7]:
laps['time'] = laps.time.apply(lambda x: format_time(str(x)))

In [34]:
# laps.to_csv(path+'data/laps.csv', index=False)

In [8]:
def lookup (df, team, points):
    df['lookup1'] = df.season.astype(str) + df[team] + df['round'].astype(str)
    df['lookup2'] = df.season.astype(str) + df[team] + (df['round']-1).astype(str)
    new_df = df.merge(df[['lookup1', points]], how = 'left', left_on='lookup2',right_on='lookup1')
    new_df.drop(['lookup1_x', 'lookup2', 'lookup1_y'], axis = 1, inplace = True)
    new_df.rename(columns = {points+'_x': points+'_after_race', points+'_y': points}, inplace = True)
    new_df[points].fillna(0, inplace = True)
    return new_df

In [9]:
for col in ['driver_points', 'driver_wins', 'driver_standings_pos']:
    drivers = lookup(drivers, 'driver', col)

drivers.drop(['driver_points_after_race', 'driver_wins_after_race', 'driver_standings_pos_after_race'], axis=1, inplace=True)

In [10]:
for col in ['constructor_points', 'constructor_wins', 'constructor_standings_pos']:
    constructors = lookup(constructors, 'constructor', col)

constructors.drop(['constructor_points_after_race', 'constructor_wins_after_race', 'constructor_standings_pos_after_race'], axis=1, inplace=True)

In [11]:
mean_time = laps.groupby(['season', 'round', 'driver']).agg({'time': 'mean'}).reset_index()
mean_time['time'] = mean_time.time.apply(lambda x: round(x, 3))

In [33]:
df1 = pd.merge(races, results, how='inner', on=['season', 'round', 'circuit_id']).drop(['url','points', 'status', 'time', 'grid'], axis=1)
df2 = pd.merge(df1, drivers, how='left', on=['season', 'round', 'driver']) 
df3 = pd.merge(df2, constructors, how='left', on=['season', 'round', 'constructor'])
df4 = pd.merge(df3, mean_time, how='left', on=['season', 'round', 'driver'])

merged_df = pd.merge(df4, qualifying, how='inner', on=['season', 'round', 'circuit_id', 'driver'])
merged_df.rename(columns = {'time': 'average_pace', 'fastest_time': 'qual_time'}, inplace=True)

merged_df.shape

(3671, 24)

In [34]:
merged_df['fastest_lap'] = merged_df['fastest_lap'].apply(lambda x: format_time(str(x)))

In [35]:
merged_df[['fastest_lap', 'qual_time', 'average_pace']].head()

,fastest_lap,qual_time,average_pace
0,92.478,104.595,97.872
1,93.066,105.745,98.342
2,92.917,104.437,98.399
3,93.186,105.819,98.491
4,92.616,108.147,98.708


In [36]:
merged_df[['season','round','driver','grid', 'podium','fastest_lap']].query('season == 2015 & round == 1').sort_values('grid')

,season,round,driver,grid,podium,fastest_lap
405,2015,1,hamilton,1,1,90.945
406,2015,1,rosberg,2,2,91.092
408,2015,1,massa,3,4,91.719
407,2015,1,vettel,4,3,91.457
416,2015,1,raikkonen,5,12,91.426
422,2015,1,bottas,6,18,NaN
410,2015,1,ricciardo,7,6,92.797
413,2015,1,sainz,8,9,92.872
418,2015,1,grosjean,9,14,NaN
419,2015,1,maldonado,10,15,NaN


In [29]:
merged_df.columns

Index(['season', 'round', 'circuit_id', 'country', 'lat', 'long', 'date',
       'driver', 'date_of_birth', 'nationality', 'constructor', 'podium',
       'fastest_lap', 'driver_points', 'driver_wins', 'driver_standings_pos',
       'constructor_points', 'constructor_wins', 'constructor_standings_pos',
       'average_pace', 'grid', 'qual_time', 'stage', 'q_delta'],
      dtype='object')

In [55]:
null_cols = merged_df.columns[merged_df.isna().any()].tolist()
# merged_df.query()
null_rows = merged_df[merged_df.isna().any(axis=1)]
null_rows = null_rows[['season', 'round', 'driver', 'podium', 'average_pace', 'qual_time','fastest_lap', 'stage']]

In [56]:
null_rows

,season,round,driver,podium,average_pace,qual_time,fastest_lap,stage
15,2014,1,maldonado,16,104.033,NaN,94.766,NaN
19,2014,1,massa,20,NaN,108.079,100.287,q3
20,2014,1,kobayashi,21,NaN,105.867,NaN,q2
21,2014,1,ricciardo,22,98.302,104.548,NaN,q3
43,2014,2,perez,22,NaN,122.511,NaN,q2
...,...,...,...,...,...,...,...,...
3609,2022,18,sainz,19,NaN,89.361,NaN,q3
3610,2022,18,albon,20,NaN,91.311,NaN,q1
3630,2022,19,sainz,20,115.016,94.356,NaN,q3
3669,2022,21,kevin_magnussen,19,NaN,71.674,NaN,q3


In [31]:
# fill/drop nulls
### Data is being lossed here

for col in ['driver_points', 'driver_wins', 'driver_standings_pos', 'constructor_points', 
            'constructor_wins', 'constructor_standings_pos']:
    merged_df[col].fillna(0, inplace=True)
    merged_df[col] = merged_df[col].map(lambda x: int(x))
    
merged_df.dropna(inplace=True)

merged_df.shape

(2990, 24)

In [18]:
from dateutil.relativedelta import *

merged_df['date_of_birth'] = pd.to_datetime(merged_df.date_of_birth)
merged_df['driver_age'] = merged_df.apply(lambda x: relativedelta(pd.to_datetime(x['date']), x['date_of_birth']).years, axis=1)
merged_df.drop(['date_of_birth'], axis=1, inplace = True)

In [19]:
merged_df.to_csv(path+'data/merged.csv', index=False)